# LSTM
* Problem Cümlemiz : LSTM ile ürün yorumları üzerinden metin türetme görevi

## Kütüphanelerin Yüklenmesi

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd 
import re
import pickle
from collections import Counter
from torch.utils.data import Dataset,DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Kullanılan Cihaz: ",device)


Kullanılan Cihaz:  cpu


## Veri Yükleme

In [5]:
df = pd.read_csv("e-ticaret_urun_yorumlari.csv",delimiter=";",encoding="utf-8")
df.head()

,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15170 entries, 0 to 15169
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Metin   15170 non-null  object
 1   Durum   15170 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 237.2+ KB


In [9]:
texts = df["Metin"].tolist()
print("Toplam Veri Sayısı:",len(texts))

Toplam Veri Sayısı: 15170


## veri temizleme

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9ıüöğçş]",' ',text) # türkçe karakterlerde dikkate alındı
    text = re.sub(r'\\s+',' ',text).strip()
    return text
# ÖDEV :  veri temizleme dökümanını inceleyerek yazım düzeltmeler için gerekli kodlar eklenebilir
temiz_metinler = [clean_text(text) for text in texts]
print("Örnek Temizlenmiş Metin:",temiz_metinler[8])
print("Temizlenmemiş Metin:",texts[8])

Örnek Temizlenmiş Metin: tavsiye edebileceğim çok güzel bir makina
Temizlenmemiş Metin: tavsiye edebileceğim çok güzel bir makina


## Tokenizasyon ve Sözlük Oluşturma

In [14]:
# kelime frekans analizi ve sözlük oluşturma 
kelimeler = " ".join(temiz_metinler).split()
frekans = Counter(kelimeler)
en_cok_kelimeler = frekans.most_common(5000)

# sozluk : Kelime -> İndis
kelime2idx = {"<PAD>":0,"<OOV>":1} # 0 : padding ve 1 : out of vocabulary
for idx,(kelime,_) in enumerate(en_cok_kelimeler,2): # 2 den başlayarak en sık kullanılan 5000 kelimeyle sözlük oluşturuyoruz
    kelime2idx[kelime] = idx

# idx2kelime : İndis -> Kelime Ters Sözlük
idx2kelime = {idx:kelime for kelime,idx in kelime2idx.items()}

toplam_kelime = len(kelime2idx)
print(f"Toplam Kelime Sayısı: {toplam_kelime}")
print(f"Örnek Eşleme: 'ürün' -> {kelime2idx.get('ürün',1)}" )

Toplam Kelime Sayısı: 5002
Örnek Eşleme: 'ürün' -> 3


In [ ]:
# kelimeler = ["evet","çok","başarılı"]
# seq = [kelime2idx.get(kelime,1) for kelime in kelimeler]
# print(seq)
# if len(seq) < 25:
#     seq = [0] * (25-len(seq)) + seq
# else:
#     seq = seq[:25] # eğer çok uzunsa kes
# seq

[227, 2, 137]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 227, 2, 137]

## Sequence Oluşturma ve Padding

In [23]:
max_len = 25 # sabit sequence uzuluğu
sequences = []

for metin in temiz_metinler:
    kelimeler = metin.split()
    seq = [kelime2idx.get(kelime,1) for kelime in kelimeler] # Bilinmeyenler için OOV kullanıldı

    if len(seq) < max_len:
        seq = [0] * (max_len-len(seq)) + seq
    else:
        seq = seq[:max_len] # eğer çok uzunsa kes
    sequences.append(seq)

X = np.array(sequences)
print("X.shape:",X.shape)
print("Örnek Sequence",X[6])

X.shape: (15170, 25)
Örnek Sequence [   0    0    0    0    0    0    0    3  246    6  160   18   45    1
    4  143   62    1  255   15 2178    7  145  156   54]


## LSTM Modelini Tanımlama

In [ ]:
class LSTMGenerator(nn.Module):
    def __init__(self,vocab_size,embedding_dim,hidden_dim,num_layers):
        super(LSTMGenerator,self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,num_layers,batch_first=True,dropout=0.2)
        self.fc = nn.Linear(hidden_dim,vocab_size)

    def forward(self,x):
        x = self.embedding(x)
        x, _ =self.lstm(x)
        x = self.fc(x)
        return x

## Hiper Parametreler

In [ ]:
hp_embedding_dim = 128 # Denge: 64 (hafif) - 128 (orta) - 256 (kuvvetli)
hp_hidden_dim = 256 # LSTM Bellek Kapasitesi 
hp_num_layers = 2 # Derinlik : 1 (hafif) - 2 (orta) - 3 (kuvvetli)  daha karmaşık kalıpları öğrenebilmesi için sayı arttırılmalı

## Modeli Oluşturma

In [ ]:
model = LSTMGenerator(
                    vocab_size=toplam_kelime,
                    embedding_dim=hp_embedding_dim,
                    hidden_dim=hp_hidden_dim,
                    num_layers=hp_hidden_dim).to(device)

print("Model Oluşturuldu")

Model Oluşturuldu


## Modelin Eğitimi

In [28]:
# Eğitim Parametreleri
criterion = nn.CrossEntropyLoss(ignore_index=0) # paddingi ignore et
optimizer = torch.optim.Adam(model.parameters(),lr=0.001) # 
epochs = 100
batch_size = 64


model.train()
for epoch in range(epochs): 
    total_loss = 0
    for i in range(0,len(X),batch_size):
        # batch hazırla
        X_batch = X[i:i+batch_size] # girdi
        y_bacth = np.copy(X_batch) # hedef bir sonraki kelime
        y_batch = np.roll(y_bacth,-1,axis=1) # hedefi kaydır
        y_bacth[:,-1] = 0 # son kelimeyi padding yap

        # Tensor'a Çevir
        X_tensor = torch.tensor(X_batch,dtype=torch.long).to(device)
        y_tensor = torch.tensor(y_batch,dtype=torch.long).to(device)

        # ileri besleme
        optimizer.zero_grad()
        outputs = model(X_tensor) # (batch seq vocab)


        loss = criterion(outputs.view(-1,toplam_kelime),y_tensor.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs},Loss:{total_loss/len(X):.4f}")
        

Epoch 1/100,Loss:0.1019
Epoch 11/100,Loss:0.0751
Epoch 21/100,Loss:0.0626
Epoch 31/100,Loss:0.0521
Epoch 41/100,Loss:0.0441
Epoch 51/100,Loss:0.0387
Epoch 61/100,Loss:0.0342
Epoch 71/100,Loss:0.0313
Epoch 81/100,Loss:0.0292
Epoch 91/100,Loss:0.0270


## Model Test

In [29]:
def yorum_uret(baslangic,max_uzunluk=20):
    model.eval()
    cumle = clean_text(baslangic)
    kelimeler = cumle.split()

    # ilk sequence oluştur
    seq = [kelime2idx.get(kelime,1) for kelime in kelimeler]
    seq = [0]*(max_len - len(seq)) + seq

    for _ in range(max_uzunluk):
        
        input_tensor = torch.tensor([seq[-max_len:]],dtype=torch.long).to(device)
        # tahmin
        with torch.no_grad():
            output = model(input_tensor)
            tahmin = output[0,-1,:].argmax().item()

        # kelimeye çevir
        tahminlenen_kelime = idx2kelime.get(tahmin,"<OOV>")

        # Dögüyü kır
        if tahminlenen_kelime in [".","!",'?',"<OOV>"] or len(kelimeler) > 50:
            break

        kelimeler.append(tahminlenen_kelime)
        seq.append(tahmin)

    return " ".join(kelimeler)

                    

## Örnek üretim

In [34]:
print("Üretilen Yorumlar:")
print("1. 'ürün' ile başla:",yorum_uret("ürün"))
print("2. 'kalitesi iyi,çok beğendim' ile başla",yorum_uret("kalitesi iyi,çok beğendim"))
print("3. 'hızlı kargo' ile başla",yorum_uret("hızlı kargo"))

Üretilen Yorumlar:
1. 'ürün' ile başla: ürün çok küçüktü ve kalitesizdi tavsiye etmiyorum iade ettim
2. 'kalitesi iyi,çok beğendim' ile başla kalitesi iyi çok beğendim tavsiye ederim ayrıca
3. 'hızlı kargo' ile başla hızlı kargo ve güvenilir alışveriş için atlamak için uygun değil beğenmedim almayın iade edeceğim beğenmedim çok kalitesiz bir


## Modeli Kaydetme

In [43]:
torch.save({
    'model_state_dict': model.state_dict(),
    'kelime2idx': kelime2idx,
    'idx2kelime': idx2kelime,
    'max_len': max_len
},
           'urun_yorumu_uret.pth')


## Modeli Yükleme

In [38]:
hp_embedding_dim = 128 # Denge: 64 (hafif) - 128 (orta) - 256 (kuvvetli)
hp_hidden_dim = 256 # LSTM Bellek Kapasitesi 
hp_num_layers = 2 # Derinlik : 1 (hafif) - 2 (orta) - 3 (kuvvetli)  daha karmaşık kalıpları öğrenebilmesi için sayı arttırılmalı

In [44]:
model = LSTMGenerator(vocab_size=toplam_kelime,embedding_dim=hp_embedding_dim,hidden_dim=hp_hidden_dim,num_layers=hp_hidden_dim).to(device)
checkpoint = torch.load('/Users/ibrahimediz/Documents/Github/DVA_10072025/DeepLearning/urun_yorumu_uret.pth')
model.load_state_dict(checkpoint['model_state_dict'])
kelime2idx = checkpoint['kelime2idx']
idx2kelime = checkpoint['idx2kelime']
max_len = checkpoint['max_len']
model.eval()

LSTMGenerator(
  (embedding): Embedding(5002, 128)
  (lstm): LSTM(128, 256, num_layers=256, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=5002, bias=True)
)